In [1]:
import numpy as np
import numpy as np
import pandas as pd
#import WindFarmGenetic_v1 # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [2]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver2_N'

In [3]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
n_mc_samples=10000 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 80  # number of cells each row
rows_cells = 80  # number of cells each column
cell_width = 50 # unit (TBD)--> have to be computed.
constraint_dist = 400 #unit distance between two turbines
constraint_perim = 50

radius=np.ceil(constraint_dist/cell_width)

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=1 #number of independent runs with random initialization.


shell_default_location='../../data/Shell_Hackathon_Dataset/'
wind_inst_freq_file=None

In [4]:
perim_cells=int(np.ceil(constraint_perim/cell_width)) #TBU generalize it for multiple boundary cells.
NA_loc=[] 

for curr_perim_cell in range(perim_cells):
    NA_loc.extend([i+cols_cells*curr_perim_cell for i in range(cols_cells)]) #upper row
    NA_loc.extend([i*(cols_cells)+curr_perim_cell for i in range(rows_cells)]) #left boundary
    NA_loc.extend([i*(cols_cells)+cols_cells-1-curr_perim_cell for i in range(rows_cells)])#right Boundary
    NA_loc.extend([(cols_cells)*(rows_cells-1-curr_perim_cell)+i for i in range(rows_cells)])#down Boundary


In [5]:
paraDict = {}
paraDict['wt_N']=wt_N  #number of turbines
paraDict['population_size']=population_size  #population size/number of layouts for genetic algorithms
paraDict['n_mc_samples']=n_mc_samples #population/layout for SVM training
paraDict['iteration_times']=iteration_times  #number of iterations

# wind farm size, cells
paraDict['cols_cells'] = cols_cells  # number of cells each row
paraDict['rows_cells'] = rows_cells  # number of cells each column
paraDict['cell_width'] = cell_width # unit (TBD)--> have to be computed.
paraDict['constraint_dist'] = constraint_dist #unit distance between two turbines
paraDict['constraint_perim'] = constraint_perim

#genetic algorithmic parameters.
paraDict['elite_rate'] =elite_rate
paraDict['cross_rate'] = cross_rate
paraDict['mutate_rate'] = mutate_rate

paraDict['random_rate'] = random_rate
paraDict['num_runs']=num_runs #number of independent runs with random initialization.

In [6]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)
fname= data_folder+"/"+"exp_specs.txt"
file = open(fname, 'w')
file.write(str(paraDict))
file.close()

In [7]:
import WindFarmGenetic_v2 as WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate,constraint_dist=constraint_dist,
                                     shell_default_location=shell_default_location, wind_inst_freq_file=wind_inst_freq_file)

Initialization Completed


In [8]:
print(np.array(NA_loc))

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73   74   75   76   77   78   79    0   80  160  240
  320  400  480  560  640  720  800  880  960 1040 1120 1200 1280 1360
 1440 1520 1600 1680 1760 1840 1920 2000 2080 2160 2240 2320 2400 2480
 2560 2640 2720 2800 2880 2960 3040 3120 3200 3280 3360 3440 3520 3600
 3680 3760 3840 3920 4000 4080 4160 4240 4320 4400 4480 4560 4640 4720
 4800 4880 4960 5040 5120 5200 5280 5360 5440 5520 5600 5680 5760 5840
 5920 6000 6080 6160 6240 6320   79  159  239  319  399  479  559  639
  719  799  879  959 1039 1119 1199 1279 1359 1439 1519 1599 1679 1759
 1839 1919 1999 2079 2159 2239 2319 2399 2479 2559 2639 2719 2799 2879
 2959 

In [9]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

Generation layout: 100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [ ]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...


layouts:   1%|          | 1/100 [00:00<00:18,  5.35it/s]

Support vector regression guided genetic algorithm starts....
generation 0...


layouts: 100%|██████████| 100/100 [00:20<00:00,  4.95it/s]


[517.49664 519.4627  517.3847  519.2348  518.96234 519.1223  518.77637
 517.2463  519.34174 517.75653 518.9183  518.09863 519.6191  519.859
 519.05347 519.1523  518.59607 518.53046 519.8028  519.42163 519.6813
 520.1351  520.5224  518.8664  520.6584  518.8003  519.6338  519.4761
 518.66693 518.85486 519.8325  518.56537 519.61993 519.5559  516.98566
 519.2649  520.08655 519.0159  519.35846 519.8645  519.3823  519.193
 518.00323 518.48883 519.3295  519.0242  517.6767  520.2678  519.14014
 518.97    518.4353  518.5012  520.3169  519.23425 517.4391  517.4664
 518.3429  518.9815  520.22424 518.1501  518.3378  519.00385 519.7399
 518.46124 520.25836 518.96375 520.138   520.35095 519.9696  520.40314
 520.24774 520.22925 519.1821  519.3961  518.4511  517.67975 518.2021
 518.8501  520.43304 520.59717 518.904   518.1564  518.9602  518.00476
 518.6565  519.1455  518.7667  518.89624 520.7193  519.0751  519.1501
 519.5588  518.219   519.8447  517.4355  518.0189  518.2192  519.0011
 519.0058  519.03

...

[520.0119  521.06647 519.6291  520.5886  520.7353  520.2775  520.33624
 519.5222  520.658   520.38446 519.93097 519.39594 519.29254 520.2232
 519.65485 520.0692  519.912   519.4403  516.5057  519.3505  519.1284
 518.4684  519.25995 518.83057 519.78925 519.94965 519.0986  517.69525
 519.85974 518.628   519.5145  519.1968  519.4026  517.7755  519.1495
 517.9663  519.4306  520.13745 518.6757  520.07416 516.93604 518.7234
 519.08984 518.3935  519.4006  519.4386  519.6064  519.4454  518.32446
 519.54767 519.6799  519.1726  519.0038  518.3589  519.7471  519.82556
 519.57715 517.0799  517.2199  519.3008  518.918   520.1191  517.3438
 518.4387  519.12854 519.52783 519.4043  518.6892  518.61786 519.82153
 519.3783  519.14355 519.9856  517.76764 517.6647  520.1404  519.59985
 519.42175 518.65216 519.9231  520.12616 518.1312  518.2359  519.7993
 518.962   518.1894  520.667   518.7618  519.9905  519.5646  520.2221
 518.9863  519.7255  520.6215  519.2874  518.23676 518.8839  520.5781
 518.81586 519

layouts:   0%|          | 0/100 [00:00<?, ?it/s]

generation 3...


layouts: 100%|██████████| 100/100 [00:26<00:00,  3.77it/s]


[519.8616  520.3282  519.34143 519.812   518.4743  519.7995  520.1762
 520.4038  519.5347  518.4606  519.8091  518.1748  516.58563 519.0242
 518.16754 519.0457  518.37897 517.5072  519.68646 519.67755 519.4034
 519.10425 519.8101  519.56165 520.419   519.4729  519.0865  518.51263
 520.3625  519.143   519.39844 520.53955 519.7713  519.2279  517.7277
 519.83246 518.47876 520.8115  517.55023 519.6504  517.4192  518.22546
 518.26324 517.49316 517.4209  516.9837  518.5665  518.4199  518.37415
 519.89655 518.5669  519.0951  518.22363 518.7453  519.60974 519.8064
 520.3388  517.68445 518.3544  519.4559  519.3678  519.08405 518.72406
 518.9037  520.45886 517.4874  519.80505 519.61127 518.79205 519.77673
 519.063   517.59015 519.20703 519.0901  520.1685  519.53516 519.41077
 518.48505 519.93695 517.2017  519.1082  519.46545 520.2342  519.41064
 518.7174  519.01575 520.1233  517.51746 520.0086  520.3968  520.5395
 519.24774 519.7163  519.64136 519.90576 520.4802  518.8427  519.50665
 516.078   5

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797